In [1]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [2]:
#pip install tensorflow==2.2.0

In [3]:
from loaders.Loaders import CSVLoader
from compoundFeaturization.rdkitFingerprints import MorganFingerprint
from featureSelection.baseFeatureSelector import LowVarianceFS
from splitters.splitters import SingletaskStratifiedSplitter
from models.kerasModels import KerasModel
from metrics.Metrics import Metric
from metrics.metricsFunctions import roc_auc_score, precision_score, accuracy_score, confusion_matrix, classification_report


import tensorflow as tf
print(tf.version.VERSION)
from tensorflow.keras.losses import binary_crossentropy

2.2.0


In [4]:
#Load Dataset
dataset = CSVLoader(dataset_path='preprocessed_dataset_wfoodb.csv', 
                    mols_field='Smiles', 
                    labels_fields='Class', 
                    id_field='ID')#, shard_size=4000)
dataset = dataset.create_dataset()
print(dataset.get_shape())

Mols_shape:  23290
Features_shape:  X not defined!
Labels_shape:  (23290,)
None


In [5]:
#Featurization
dataset = MorganFingerprint().featurize(dataset)
dataset.get_shape()

Featurizing datapoint 0
Featurizing datapoint 1000
Featurizing datapoint 2000
Featurizing datapoint 3000
Featurizing datapoint 4000
Featurizing datapoint 5000
Featurizing datapoint 6000


RDKit ERROR: [09:22:46] Explicit valence for atom # 1 Cl, 4, is greater than permitted


error in smile: O=[Cl]=O
Featurizing datapoint 7000


RDKit ERROR: [09:22:49] Explicit valence for atom # 3 B, 4, is greater than permitted
RDKit ERROR: [09:22:49] Explicit valence for atom # 1 Cl, 9, is greater than permitted


error in smile: OB1O[B]2(O)OB(O)O[B](O)(O1)O2
error in smile: O=[Cl-](=O)(=O)=O
Featurizing datapoint 8000
Featurizing datapoint 9000
Featurizing datapoint 10000
Featurizing datapoint 11000
Featurizing datapoint 12000
Featurizing datapoint 13000
Featurizing datapoint 14000


RDKit ERROR: [09:23:05] Explicit valence for atom # 0 P, 11, is greater than permitted


error in smile: [P](OCC=C(C)C)(OCC=C(C)C)(=O)(OP(OCC=C(C)C)(OCC=C(C)C)=O)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)CC=C(C)C
Featurizing datapoint 15000
Featurizing datapoint 16000
Featurizing datapoint 17000
Featurizing datapoint 18000
Featurizing datapoint 19000
Featurizing datapoint 20000
Featurizing datapoint 21000
Featurizing datapoint 22000
Featurizing datapoint 23000
Elements with indexes:  [6257, 7708, 7709, 14244]  were removed due to the presence of NAs!
The elements in question are:  ['O=[Cl]=O' 'OB1O[B]2(O)OB(O)O[B](O)(O1)O2' 'O=[Cl-](=O)(=O)=O'
 '[P](OCC=C(C)C)(OCC=C(C)C)(=O)(OP(OCC=C(C)C)(OCC=C(C)C)=O)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)CC=C(C)C']
Mols_shape:  23286
Features_shape:  (23286, 1024)
Labels_shape:  (23286,)


In [6]:
#Feature Selection
dataset = LowVarianceFS(0.15).featureSelection(dataset)
dataset.get_shape()

Mols_shape:  23286
Features_shape:  (23286, 49)
Labels_shape:  (23286,)


In [7]:
#Data Split
splitter = SingletaskStratifiedSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset=dataset, frac_train=0.6, 
                                                                             frac_valid=0.2, frac_test=0.2)

In [8]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np


input_dim = train_dataset.X.shape[1]


def create_model(optimizer='adam', dropout=0.5, input_dim=input_dim):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=input_dim, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


In [9]:
#print(type(keras_model))
#from sklearn.model_selection import GridSearchCV 
#model = KerasClassifier(build_fn=create_model, epochs=15, batch_size=10, verbose=1, optimizer='adam')

#model.fit(train_dataset.features, train_dataset.y)
#params={'optimizer':['adam', 'rsmprop'],  
#        'nb_epoch':[200, 100, 300, 400]}

#gs=GridSearchCV(estimator=model, param_grid=params, cv=3) 
# now fit the dataset to the GridSearchCV object.  
#gs = gs.fit(train_dataset.features, train_dataset.y)

#print(gs)
#print(type(model))

#seed = 7
#np.random.seed(seed)
#print(1)
#kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
#print(2)
#results = cross_val_score(model, train_dataset.features, train_dataset.y, cv=kfold)
#print(3)
#print(results)

#model = KerasModel(keras_model, binary_crossentropy)

In [10]:
from models.kerasModels import KerasModel

input_dim = train_dataset.X.shape[1]
print(input_dim)
model = KerasModel(create_model, epochs = 150, verbose=1, optimizer='adam')#, input_dim=input_dim)



49


In [11]:
print(train_dataset.X.shape, train_dataset.y.shape)


model.fit(train_dataset)#.features, train_dataset.y)#, nb_epoch=50)


(13968, 49) (13968,)
kerasModels fit!
Epoch 1/150
1397/1397 [==============================] - 2s 1ms/step - loss: 0.2273 - accuracy: 0.9406
Epoch 2/150
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1859 - accuracy: 0.9434
Epoch 3/150
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1762 - accuracy: 0.9436
Epoch 4/150
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1746 - accuracy: 0.9439
Epoch 5/150
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1725 - accuracy: 0.9439
Epoch 6/150
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1697 - accuracy: 0.9447
Epoch 7/150
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1698 - accuracy: 0.9440
Epoch 8/150
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1677 - accuracy: 0.9453
Epoch 9/150
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1681 - accuracy: 0.9450
Epoch 10/150
1397/1397 [===========

1397/1397 [==============================] - 2s 1ms/step - loss: 0.1517 - accuracy: 0.9471
Epoch 79/150
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1545 - accuracy: 0.9454
Epoch 80/150
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1524 - accuracy: 0.9477
Epoch 81/150
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1518 - accuracy: 0.9470
Epoch 82/150
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1547 - accuracy: 0.9463
Epoch 83/150
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1541 - accuracy: 0.9458
Epoch 84/150
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1528 - accuracy: 0.9472
Epoch 85/150
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1518 - accuracy: 0.9461
Epoch 86/150
1397/1397 [==============================] - 2s 1ms/step - loss: 0.1520 - accuracy: 0.9471
Epoch 87/150
1397/1397 [==============================] - 2s 1ms/step - loss:

In [12]:
metrics = [Metric(roc_auc_score), 
           Metric(precision_score), 
           Metric(accuracy_score), 
           Metric(confusion_matrix), 
           Metric(classification_report)]

print('training set score:', model.evaluate(train_dataset, metrics))
print('test set score:', model.evaluate(test_dataset, metrics))

Models Class line 158 --> evaluator
Instructions for updating:
Please use `model.predict()` instead.
1397/1397 [==============================] - 1s 780us/step
roc_auc_score: 
 0.5676714739371911
precision_score: 
 0.8571428571428571
accuracy_score: 
 0.949885452462772
confusion_matrix: 
 [[13160    18]
 [  682   108]]
classification_report: 
               precision    recall  f1-score   support

           0       0.95      1.00      0.97     13178
           1       0.86      0.14      0.24       790

    accuracy                           0.95     13968
   macro avg       0.90      0.57      0.60     13968
weighted avg       0.95      0.95      0.93     13968

training set score: {'roc_auc_score': 0.5676714739371911, 'precision_score': 0.8571428571428571, 'accuracy_score': 0.949885452462772, 'confusion_matrix': 3492.0, 'classification_report': None}
Models Class line 158 --> evaluator
466/466 [==============================] - 0s 767us/step
roc_auc_score: 
 0.5495425567146879
preci

In [13]:
from parameterOptimization.HyperparameterOpt import GridHyperparamOpt
#Hyperparameter Optimization
optimizer = GridHyperparamOpt(create_model)

params_dict = {'optimizer' : ['adam', 'rmsprop'],
              'dropout' : [0.2, 0.4, 0.5]}

best_model, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict, train_dataset, 
                                                                        valid_dataset, Metric(roc_auc_score),
                                                                       cv=5, n_jobs=1, verbose=3)

print('#################')
print(best_hyperparams)
print(best_model)

Fitting 6 random models from a space of 6 possible models.
Fitting model 1/6
hyperparameters: {'optimizer': 'adam', 'dropout': 0.2}
<class 'metrics.Metrics.Metric'>
METRIC:  <metrics.Metrics.Metric object at 0x7f9c8ad95e48>
Using accuracy instead and  <metrics.Metrics.Metric object at 0x7f9c8ad95e48>  on validation!
 

(13968, 49) 2793.6
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] dropout=0.2, optimizer=adam .....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


350/350 [==============================] - 0s 1ms/step - loss: 0.2366 - accuracy: 0.9429
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[CV] ......... dropout=0.2, optimizer=adam, score=0.943, total=   0.9s
[CV] dropout=0.2, optimizer=adam .....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


350/350 [==============================] - 0s 1ms/step - loss: 0.2993 - accuracy: 0.8985
[CV] ......... dropout=0.2, optimizer=adam, score=0.943, total=   0.9s
[CV] dropout=0.2, optimizer=adam .....................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.8s remaining:    0.0s


350/350 [==============================] - 0s 1ms/step - loss: 0.2581 - accuracy: 0.9183
[CV] ......... dropout=0.2, optimizer=adam, score=0.943, total=   0.9s
[CV] dropout=0.2, optimizer=adam .....................................
350/350 [==============================] - 0s 1ms/step - loss: 0.2457 - accuracy: 0.9340
[CV] ......... dropout=0.2, optimizer=adam, score=0.943, total=   0.9s
[CV] dropout=0.2, optimizer=adam .....................................
350/350 [==============================] - 0s 1ms/step - loss: 0.2841 - accuracy: 0.9213
[CV] ......... dropout=0.2, optimizer=adam, score=0.943, total=   0.9s
[CV] dropout=0.2, optimizer=rmsprop ..................................
350/350 [==============================] - 1s 2ms/step - loss: 0.2215 - accuracy: 0.9428
[CV] ...... dropout=0.2, optimizer=rmsprop, score=0.943, total=   1.1s
[CV] dropout=0.2, optimizer=rmsprop ..................................
350/350 [==============================] - 1s 2ms/step - loss: 0.2277 - accu

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   29.7s finished


437/437 [==============================] - 1s 1ms/step - loss: 0.3456 - accuracy: 0.8668

 
 Best accuracy: 0.943442 using {'dropout': 0.2, 'optimizer': 'adam'}

 accuracy: 0.943442 (0.000010) with: {'dropout': 0.2, 'optimizer': 'adam'} 


 accuracy: 0.943442 (0.000010) with: {'dropout': 0.2, 'optimizer': 'rmsprop'} 


 accuracy: 0.943442 (0.000010) with: {'dropout': 0.4, 'optimizer': 'adam'} 


 accuracy: 0.943442 (0.000010) with: {'dropout': 0.4, 'optimizer': 'rmsprop'} 


 accuracy: 0.943442 (0.000010) with: {'dropout': 0.5, 'optimizer': 'adam'} 


 accuracy: 0.943442 (0.000010) with: {'dropout': 0.5, 'optimizer': 'rmsprop'} 

#################
{'dropout': 0.2, 'optimizer': 'adam'}


In [14]:
#pip install tensorflow==2.2.0

In [15]:

#import tensorflow as tf
#tf.version.VERSION

In [16]:
import sklearn
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])